# Installs

## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
!pip install wandb -q

In [ ]:
import wandb
wandb.login(key="a8b39d27e28590586b7efa4abb3acaad4e91b958")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: akh. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# config = {
#     'epochs': 25,
#     'batch_size' : 1024,
#     'learning_rate' : 0.001, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
#     # Include other parameters as needed.
# }

## Levenshtein

This may take a while

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX

## imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!nvidia-smi

Fri Nov 18 01:58:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    12W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"almutwakelhassan","key":"*****************"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c 11-785-f22-hw3p2

In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-f22-hw3p2.zip
!ls

# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, data_path='/content/hw3p2/train-clean-360/', partition="train", limit=1): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = data_path + 'mfcc/'
        
        self.transcript_dir = data_path + 'transcript/raw' 

        mfcc_files = sorted(os.listdir(self.mfcc_dir))
        transcript_files = sorted(os.listdir(self.transcript_dir))

        # mfcc_files.remove("processed.csv")
        # mfcc_files.remove("raw")

        self.PHONEMES = PHONEMES

        num_files = len(mfcc_files)
        if partition == "train":
          mfcc_files = mfcc_files[0:int(num_files * limit)]
          transcript_files = transcript_files[0:int(num_files * limit)]
        elif partition == "validate":
          mfcc_files = mfcc_files[int(num_files * limit) + 1:]
          transcript_files = transcript_files[int(num_files * limit) + 1:]


        assert len(mfcc_files) == len(transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []
          

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_files)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_files[i]), allow_pickle=True)
        #   Optionally do Cepstral Normalization of mfcc
            mfcc = (mfcc - np.mean(mfcc, axis=0))/(np.std(mfcc, axis=0)) # edit?

        #   Load the corresponding transcript
            transcript = np.load(os.path.join(self.transcript_dir, transcript_files[i]), allow_pickle=True)[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfccs)
        # self.transcripts = np.vectorize(self.PHONEMES.index)(self.transcripts)
        self.transcripts = [[self.PHONEMES.index(t) for t in line] for line in self.transcripts]

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        
        mfcc = torch.FloatTensor(self.mfccs[ind]) # TODO
        transcript = torch.Tensor(self.transcripts[ind]) # TODO
        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [i for i, j in batch]
        # batch of output phonemes
        batch_transcript = [j for i, j in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO 

        batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(batch_transcript, batch_first=True) # TODO
        lengths_transcript = [len(i) for i in batch_transcript] # TODO
        

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

In [ ]:
class AudioDataset2(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, data_path='/content/hw3p2/train-clean-360/', partition="train", limit=1): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = data_path + 'mfcc/'
        
        self.transcript_dir = data_path + 'transcript/raw' 

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        # mfcc_files.remove("processed.csv")
        # mfcc_files.remove("raw")

        self.PHONEMES = PHONEMES

        # num_files = len(mfcc_files)
        # if partition == "train":
        #   mfcc_files = mfcc_files[0:int(num_files * limit)]
        #   transcript_files = transcript_files[0:int(num_files * limit)]
        # elif partition == "validate":
        #   mfcc_files = mfcc_files[int(num_files * limit) + 1:]
        #   transcript_files = transcript_files[int(num_files * limit) + 1:]


        # assert len(mfcc_files) == len(transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []
          

        # TODO:
        # Iterate through mfccs and transcripts
        # for i in range(0, len(mfcc_files)):
        

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        # self.transcripts = np.vectorize(self.PHONEMES.index)(self.transcripts)
        # self.transcripts = [[self.PHONEMES.index(t) for t in line] for line in self.transcripts]

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
    #   Load a single mfcc
        mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[ind]), allow_pickle=True)
    #   Optionally do Cepstral Normalization of mfcc
        mfcc = (mfcc - np.mean(mfcc, axis=0))/(np.std(mfcc, axis=0)) # edit?

    #   Load the corresponding transcript
        transcript = np.load(os.path.join(self.transcript_dir, self.transcript_files[ind]), allow_pickle=True)[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
        transcript = [self.PHONEMES.index(t) for t in transcript]
        # without traversing through the transcript?)
    #   Append each mfcc to self.mfcc, transcript to self.transcript
        # self.mfccs.append(mfcc)
        # self.transcripts.append(transcript)
        # mfcc = torch.FloatTensor(self.mfccs[ind]) # TODO
        mfcc = torch.FloatTensor(mfcc)
        transcript = torch.Tensor(transcript)
        # transcript = torch.Tensor(self.transcripts[ind]) # TODO
        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [i for i, j in batch]
        # batch of output phonemes
        batch_transcript = [j for i, j in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO 

        batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(batch_transcript, batch_first=True) # TODO
        lengths_transcript = [len(i) for i in batch_transcript] # TODO
        

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, data_path='/content/hw3p2/test-clean/'): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = data_path + 'mfcc/'
        
        # self.transcript_dir = data_path + 'transcript/raw' 

        mfcc_files = sorted(os.listdir(self.mfcc_dir))
        # transcript_files = sorted(os.listdir(self.transcript_dir))

        # mfcc_files.remove("processed.csv")
        # mfcc_files.remove("raw")

        self.PHONEMES = PHONEMES

        num_files = len(mfcc_files)
        # if partition == "train":
          # mfcc_files = mfcc_files[0:int(num_files * limit)]
          # transcript_files = transcript_files[0:int(num_files * limit)]
        # elif partition == "validate":
          # mfcc_files = mfcc_files[int(num_files * limit) + 1:]
          # transcript_files = transcript_files[int(num_files * limit) + 1:]


        # assert len(mfcc_files) == len(transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []
          

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_files)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_files[i]), allow_pickle=True)
        #   Optionally do Cepstral Normalization of mfcc
            mfcc = (mfcc - np.mean(mfcc, axis=0))/(np.std(mfcc, axis=0)) # edit?

        #   Load the corresponding transcript
            # transcript = np.load(os.path.join(self.transcript_dir, transcript_files[i]), allow_pickle=True)[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            # self.transcripts.append(transcript)

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfccs)
        # self.transcripts = np.vectorize(self.PHONEMES.index)(self.transcripts)
        # self.transcripts = [[self.PHONEMES.index(t) for t in line] for line in self.transcripts]

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        
        mfcc = torch.FloatTensor(self.mfccs[ind]) # TODO
        # transcript = torch.FloatTensor(self.transcripts[ind]) # TODO
        return mfcc #, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = batch
        # batch of output phonemes
        # batch_transcript = [j for i, j in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO 

        # batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(batch_transcript, batch_first=True) # TODO
        # lengths_transcript = [len(i) for i in batch_transcript_pad] # TODO
        

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

       

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2' 

### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

253

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset() #TODO
val_data = AudioDataset(data_path='/content/hw3p2/dev-clean/') # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest() #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=BATCH_SIZE, pin_memory= True,
                                           shuffle= True, collate_fn= train_data.collate_fn)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=BATCH_SIZE, pin_memory= True,
                                         shuffle= False, collate_fn= val_data.collate_fn)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=BATCH_SIZE, pin_memory= True, 
                                          shuffle= False, collate_fn= test_data.collate_fn)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 104014, batches = 1626
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 
# torch.Size([64, 1678, 15]) torch.Size([64, 212]) torch.Size([64]) torch.Size([64])

torch.Size([64, 1679, 15]) torch.Size([64, 197]) torch.Size([64]) torch.Size([64])


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([64, 1691, 15]) torch.Size([64, 198]) torch.Size([64]) torch.Size([64])


# Model Config

In [ ]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

## Basic

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = 15, hidden_size = 256, num_layers = 1, batch_first=True, bidirectional=True) 
        # hidden_size = 256

        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            torch.nn.Linear(512, OUT_SIZE)
        )        
        self.logSoftmax = torch.nn.LogSoftmax(dim=2)
        #TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        packed = torch.nn.utils.rnn.pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)
        packed, (x, y) = self.lstm.forward(packed)
        seq_unpacked, lens_unpacked = torch.nn.utils.rnn.pad_packed_sequence(packed, batch_first=True)
        z = self.classification.forward(seq_unpacked)
        z = self.logSoftmax.forward(z)
        return z, lens_unpacked


In [ ]:
torch.cuda.empty_cache()

class CNN_LSTM_Model(nn.Module):

    def __init__(self):
        super(CNN_LSTM_Model, self).__init__()
        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.feature_extraction = torch.nn.Sequential(
            torch.nn.Conv1d(15, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm1d(256),
            torch.nn.GELU(),

            torch.nn.Conv1d(256, 512, kernel_size=3, padding=1),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),

            torch.nn.Conv1d(512, 512, kernel_size=3, padding=1),
            torch.nn.AvgPool1d(kernel_size=3, stride=2)
        )
        self.lstm = nn.LSTM(input_size = 512, hidden_size = 1024, num_layers = 2, 
                            dropout=0.5, batch_first=True, bidirectional=True) 
        # self.lstm2 = nn.LSTM(input_size = 512, hidden_size = 256, num_layers = 1, batch_first=True, bidirectional=True) 


        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            
            torch.nn.Linear(2048, 2048),
            torch.nn.GELU(),
            torch.nn.Dropout(0.3),
            
            torch.nn.Linear(2048, 1024),
            torch.nn.GELU(),
            torch.nn.Dropout(0.3),

            torch.nn.Linear(1024, 512),
            torch.nn.GELU(),
            torch.nn.Linear(512, OUT_SIZE)

        )        
        self.logSoftmax = torch.nn.LogSoftmax(dim=2)
        #TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        extraction = self.feature_extraction(x.permute(0, 2, 1))
        extraction = extraction.permute(0, 2, 1)

        lx = lx.clamp(max=extraction.shape[1])

        packed = torch.nn.utils.rnn.pack_padded_sequence(extraction, lx, batch_first=True, enforce_sorted=False)
        packed1, (x, y) = self.lstm(packed)
        seq_unpacked, lens_unpacked = torch.nn.utils.rnn.pad_packed_sequence(packed1, batch_first=True)
        z = self.classification(seq_unpacked)
        z = self.logSoftmax(z)
        
        return z, lens_unpacked


## INIT

In [ ]:
torch.cuda.empty_cache()
gc.collect()

model = CNN_LSTM_Model().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                                     Kernel Shape     Output Shape  \
Layer                                                                
0_feature_extraction.Conv1d_0        [15, 256, 3]  [64, 256, 1691]   
1_feature_extraction.BatchNorm1d_1          [256]  [64, 256, 1691]   
2_feature_extraction.GELU_2                     -  [64, 256, 1691]   
3_feature_extraction.Conv1d_3       [256, 512, 3]  [64, 512, 1691]   
4_feature_extraction.BatchNorm1d_4          [512]  [64, 512, 1691]   
5_feature_extraction.GELU_5                     -  [64, 512, 1691]   
6_feature_extraction.Conv1d_6       [512, 512, 3]  [64, 512, 1691]   
7_feature_extraction.AvgPool1d_7                -   [64, 512, 845]   
8_lstm                                          -    [52953, 2048]   
9_classification.Linear_0            [2048, 2048]  [64, 845, 2048]   
10_classification.GELU_1                        -  [64, 845, 2048]   
11_classification.Dropout_2                     -  [64, 845, 2048]   
12_classification.Li

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_feature_extraction.Conv1d_0,"[15, 256, 3]","[64, 256, 1691]",11776.0,1.948032e+07
1_feature_extraction.BatchNorm1d_1,[256],"[64, 256, 1691]",512.0,2.560000e+02
2_feature_extraction.GELU_2,-,"[64, 256, 1691]",NaN,NaN
3_feature_extraction.Conv1d_3,"[256, 512, 3]","[64, 512, 1691]",393728.0,6.649283e+08
4_feature_extraction.BatchNorm1d_4,[512],"[64, 512, 1691]",1024.0,5.120000e+02
5_feature_extraction.GELU_5,-,"[64, 512, 1691]",NaN,NaN
6_feature_extraction.Conv1d_6,"[512, 512, 3]","[64, 512, 1691]",786944.0,1.329857e+09
7_feature_extraction.AvgPool1d_7,-,"[64, 512, 845]",NaN,NaN
8_lstm,-,"[52953, 2048]",37781504.0,3.774874e+07


# Training Config

In [ ]:
train_config = {
    "beam_width" : 3,
    "lr" : 1e-3,
    "epochs" : 100
    } # Feel free to add more items here

In [ ]:
run = wandb.init(
    name = "CNN_LSTM_Separate002", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = train_config ### Wandb Config for your run
)

In [ ]:
train_config

{'beam_width': 3, 'epochs': 100, 'lr': 0.001}

In [ ]:
#TODO
criterion = torch.nn.CTCLoss(reduction = 'mean')
# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=train_config['lr']) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = ctcdecode.CTCBeamDecoder(
    labels = ARPAbet,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=train_config['beam_width'],
    num_processes=4,
    blank_id=0,
    log_probs_input=True)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, factor=0.25)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["epochs"] * len(train_loader))

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

### Levenshtein

In [ ]:
# Use debug = True to see debug outputs
def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):

    if debug:
        pass
        # print(f"\n----- IN LEVENSHTEIN -----\n")
        # Add any other debug statements as you may need
        # you may want to use debug in several places in this function
        
    # TODO: look at docs for CTC.decoder and find out what is returned here
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(h, seq_lens = lh)
    # print(beam_results, beam_scores, timesteps, out_lens)
    batch_size = h.shape[0] # TODO
    distance = 0 # Initialize the distance to be 0 initially

    for i in range(batch_size): 
        # TODO: Loop through each element in the batch
        h_slice = beam_results[i, 0, :out_lens[i, 0]]
        h_string = [labels[x] for x in h_slice]
        y_slice = y[i, :ly[i]]
        
        y_string = [labels[int(x)] for x in y_slice]

        distance += Levenshtein.distance("".join(h_string), "".join(y_string))

        del h_slice, h_string, y_slice, y_string
    del beam_results, beam_scores, timesteps, out_lens

    distance /= batch_size # TODO: Uncomment this, but think about why we are doing this

    return distance

In [ ]:
# ANOTEHR SANITY CHECK

with torch.no_grad():
  for i, data in enumerate(train_loader):
      
      #TODO: 
      # Follow the following steps, and 
      # Add some print statements here for sanity checking
      # print(data)
      h, y, lh, ly = data
      h = h.to(device)
      y = y.to(device)
      # lh.to(device)
      # ly.to(device)
      # print("h", h.shape)
      # print("y", y.shape)
      # print("lh", lh.shape)
      # print("ly", ly.shape)

      out, out_len = model(h, lh)
      # print("out", out.shape)
      # print("out_len", out_len.shape)
      distance = calculate_levenshtein(out, y, out_len, ly, decoder, LABELS)
      loss = criterion(out.permute(1, 0, 2), y, out_len, ly)

      # print(make_output(out, out_len, decoder, LABELS))
      # print(make_output(y, ly, decoder, LABELS))
      #1. What values are you returning from the collate function
      #2. Move the features and target to <DEVICE>
      #3. Print the shapes of each to get a fair understanding 
      #4. Pass the inputs to the model
            # Think of the following before you implement:
            # 4.1 What will be the input to your model?
            # 4.2 What would the model output?
            # 4.3 Print the shapes of the output to get a fair understanding 

      # Calculate loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
      # Calculating the loss is not straightforward. Check the input format of each parameter

      print(f"lev-distance: {distance}")
      print(f"loss: {loss}")


      break # one iteration is enough

lev-distance: 301.546875
loss: 20.174972534179688


# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
def evaluate(data_loader, model):
    model.eval()
    dist = 0
    loss = 0
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, leave=False, position=0, desc='Val') 
    # TODO Fill this function out, if you're using it.
    ## Forgot this
    with torch.no_grad():
      for i, data in enumerate(data_loader):

          # TODO: Fill this with the help of your sanity check
          h, y, lh, ly = data
          h = h.to(device)
          y = y.to(device)
          # ly = ly.to(device)

          out, out_len = model(h, lh)
          loss += criterion(out.permute(1, 0, 2), y, out_len, ly)
          dist += calculate_levenshtein(out, y, out_len, ly, decoder, LABELS)

          del h, y, lh, ly, data, out, out_len
          torch.cuda.empty_cache()
          # torch.clear_autocast_cache()
          gc.collect()

          batch_bar.set_postfix(
              loss = f"{loss/ (i+1):.4f}",
              dist = f"{dist/(i+1):.4f}",
              lr = f"{optimizer.param_groups[0]['lr']}"
          )
          batch_bar.update()
      batch_bar.close()

    return dist/len(data_loader) ,loss/len(data_loader),

### Training Setup

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = train_config['epochs'] #epochs
best_val_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
dist_freq = 1

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:
def train_step(train_loader, model, optimizer, criterion, scheduler, scaler):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0

    for i, data in enumerate(train_loader):

        # TODO: Fill this with the help of your sanity check

        h, y, lh, ly = data
        h = h.to(device)
        y = y.to(device)
        # lh.to(device)
        # ly.to(device)
        optimizer.zero_grad()

        # HINT: Are you using mixed precision?
        
        with torch.cuda.amp.autocast():
            out, out_len = model.forward(h, lh)
            loss = criterion(out.permute(1, 0, 2), y, out_len, ly)
        # loss.backward()
        
        # print(loss.item())
        # h.to('cpu')
        # y.to('cpu')
        # ly.to('cpu')
        

        batch_bar.set_postfix(
            tloss = f"{train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        train_loss += loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        # optimizer.step()
        batch_bar.update()
        
        del h, y, lh, ly
    
    batch_bar.close()
    train_loss /= len(train_loader) # TODO

    return train_loss # And anything else you may wish to get out of this function

### Train Loop

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(train_config["epochs"]):

    # one training step
    # one validation step (if you want)

    # HINT: Calculating levenshtein distance takes a long time. Do you need to do it every epoch?
    # Does the training step even need it? 

    # Where you have your scheduler.step depends on the scheduler you use.
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, scaler)
    val_dist, val_loss = evaluate(val_loader, model)
    print(f"\nEpoch {epoch} / Val Distance {val_dist} / Train Loss {train_loss}")
    # Use the below code to save models
    if val_dist < best_val_dist:
      #path = os.path.join(root_path, model_directory, 'checkpoint' + '.pth')
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'val_dist': val_dist, 
                  'epoch': epoch}, './checkpoint.pth')
      best_val_dist = val_dist
      # wandb.save('checkpoint.pth')

    # scheduler.step(val_dist)
    
    curr_lr = float(optimizer.param_groups[0]['lr'])
    # You may want to log some hyperparameters and results on wandb
    # wandb.log({"train_loss":train_loss, 'val_dist': val_dist, "learning_Rate": curr_lr})

run.finish()


Epoch 0 / Val Distance 11.109883720930233 / Train Loss 1.3558909893035889
Saving model



Epoch 1 / Val Distance 8.447625968992247 / Train Loss 0.43633466958999634
Saving model



Epoch 2 / Val Distance 7.211627906976744 / Train Loss 0.35278111696243286
Saving model



Epoch 3 / Val Distance 6.621608527131784 / Train Loss 0.3165063261985779
Saving model



Epoch 4 / Val Distance 6.0418604651162795 / Train Loss 0.2856194078922272
Saving model


...


Epoch 80 / Val Distance 3.211967054263566 / Train Loss 0.0054629179649055
Saving model



Epoch 81 / Val Distance 3.2480135658914726 / Train Loss 0.005189976654946804



Epoch 82 / Val Distance 3.23718507751938 / Train Loss 0.004766519647091627


Train:  22%|██▏       | 365/1626 [07:37<26:26,  1.26s/it, lr=6.784478222989877e-05, tloss=0.0046]

KeyboardInterrupt: ignored

# Generate Predictions and Submit to Kaggle

In [ ]:
torch.cuda.empty_cache()
gc.collect()
!nvidia-smi

In [ ]:
torch.cuda.empty_cache()
gc.collect()
evaluate(val_loader, model)

In [ ]:
torch.cuda.empty_cache()
gc.collect()
evaluate(val_loader, model)

(3.2595687984496124, tensor(0.3999, device='cuda:0'))

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

decoder_test = CTCBeamDecoder(
    ARPAbet,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=train_config['beam_width'],
    num_processes=4,
    blank_id=0,
    log_probs_input=True)

def make_output(h, lh, decoder, LABELS):
  
    beam_results, beam_scores, timesteps, out_seq_len = decoder_test.decode(h, lh) #TODO: What parameters would the decode function take in?
    batch_size = h.shape[0] #What is the batch size

    preds = []
    for i in range(batch_size): # Loop through each element in the batch

        h_slice = beam_results[i, 0, :out_seq_len[i, 0]]
        h_string = [LABELS[x] for x in h_slice]
        preds.append("".join(h_string))
    
    return preds

In [ ]:
def predict(dataloader, model, decoder, LABELS):
    model.eval()

    test_results = []
    with torch.no_grad():
      for i, data in enumerate(dataloader):
          h, lh = data
          h = h.to(device)

          # HINT: Are you using mixed precision?

          with torch.cuda.amp.autocast():
              out, out_len = model(h, lh)
          pred = make_output(out, out_len, decoder_test, LABELS)
          test_results.extend(pred)
          del h, lh, out, out_len
          torch.cuda.empty_cache()
          gc.collect()
    
    return test_results

In [ ]:
#TODO:
# Write a function (predict) to generate predictions and submit the file to Kaggle

torch.cuda.empty_cache()
predictions = predict(test_loader, model, decoder_test, LABELS)
import pandas as pd

df = pd.read_csv('/content/hw3p2/test-clean/transcript/random_submission.csv')
df.label = predictions

df.to_csv('submission.csv', index = False)
!kaggle competitions submit -c '11-785-f22-hw3p2' -f submission.csv -m "I made it!"

100% 210k/210k [00:00<00:00, 965kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)